In [2]:
# =============================================
# STEP 1: Load and Validate Raw ESG Dataset
# =============================================
import pandas as pd

# Load raw data (no header)
df = pd.read_csv("company_esg_financial_dataset.csv", header=None)

# Assign column names 
columns = [
    "company_id",
    "company_name",
    "industry",
    "region",
    "year",
    "revenue",
    "profit_margin",
    "market_cap",
    "growth_rate",
    "esg_overall",
    "esg_environmental",
    "esg_social",
    "esg_governance",         
    "carbon_emissions",
    "water_usage",
    "energy_consumption"
]

df.columns = columns

# Convert to numeric (safe)
numeric_cols = [
    "year", "revenue", "profit_margin", "market_cap", "growth_rate",
    "esg_overall", "esg_environmental", "esg_social", "esg_governance",
    "carbon_emissions", "water_usage", "energy_consumption"
]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Show first 5 rows
print(" Data loaded successfully!")
print("\nFirst 5 rows:")
df.head()

 Data loaded successfully!

First 5 rows:


,company_id,company_name,industry,region,year,revenue,profit_margin,market_cap,growth_rate,esg_overall,esg_environmental,esg_social,esg_governance,carbon_emissions,water_usage,energy_consumption
0,CompanyID,CompanyName,Industry,Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Company_1,Retail,Latin America,2015.0,459.2,6.0,337.5,NaN,57.0,60.7,33.5,76.8,35577.4,17788.7,71154.7
2,1,Company_1,Retail,Latin America,2016.0,473.8,4.6,366.6,3.2,56.7,58.9,32.8,78.5,37314.7,18657.4,74629.4
3,1,Company_1,Retail,Latin America,2017.0,564.9,5.2,313.4,19.2,56.5,57.6,34.0,77.8,45006.4,22503.2,90012.9
4,1,Company_1,Retail,Latin America,2018.0,558.4,4.3,283.0,-1.1,58.0,62.3,33.4,78.3,42650.1,21325.1,85300.2


In [3]:
# =============================================
# STEP 2: Add Japan Context & Sort Data
# =============================================

# Treat "Asia" as Japan-relevant (for Accenture Japan focus)
df['is_japan_relevant'] = (df['region'] == 'Asia').astype(int)

# Sort by company and year (critical for trend calculations)
df = df.sort_values(['company_name', 'year']).reset_index(drop=True)

print(" Added Japan relevance flag and sorted data.")
print(f"\nTotal rows: {len(df)}")
print(f"Years covered: {df['year'].min()} to {df['year'].max()}")
print(f"Industries: {df['industry'].nunique()}")
print(f"Regions: {list(df['region'].unique())}")

 Added Japan relevance flag and sorted data.

Total rows: 11001
Years covered: 2015.0 to 2025.0
Industries: 10
Regions: ['Region', 'Latin America', 'North America', 'Oceania', 'Africa', 'Europe', 'Asia', 'Middle East']


In [4]:
# STEP 3: Advanced ESG Risk Scoring Model
# =============================================

# Ensure data is sorted 
df = df.sort_values(['company_name', 'year']).reset_index(drop=True)

In [5]:
# A1: Calculate year-over-year changes
df['esg_trend'] = df.groupby('company_name')['esg_overall'].diff()
df['profit_trend'] = df.groupby('company_name')['profit_margin'].diff()

In [6]:
# A2: ESG volatility (3-year rolling std)
df['esg_volatility'] = df.groupby('company_name')['esg_overall'].transform(
    lambda x: x.rolling(window=3, min_periods=2).std()
)

In [7]:
# A3: Industry percentile rank (0 = worst, 1 = best)
df['esg_percentile'] = df.groupby(['industry', 'year'])['esg_overall'].rank(pct=True)

# A4: Profit-ESG divergence (greenwashing signal)
df['profit_esg_divergence'] = (
    (df['esg_trend'] < 0) & (df['profit_trend'] > 0)
).astype(int)

In [9]:
# A5: Market cap volatility (proxy for investor confidence)
df['market_cap_volatility'] = df.groupby('company_name')['market_cap'].transform(
    lambda x: x.rolling(3, min_periods=2).std() / x.rolling(3, min_periods=2).mean()
)

# A6: Financial stress flag
df['financial_stress'] = (
    (df['profit_trend'] < 0) & (df['market_cap_volatility'].fillna(0) > 0.1)
).astype(int)


In [10]:
# A7: Final Risk Score (0–100 scale)
# Higher = higher risk
df['risk_score'] = (
    0.25 * (1 - df['esg_percentile']) * 100 +          # Bottom performers penalized
    0.20 * df['esg_volatility'].fillna(0) * 10 +        # Volatility = risk
    0.20 * df['profit_esg_divergence'] * 15 +           # Greenwashing signal
    0.20 * df['financial_stress'] * 20 +                # Financial red flag
    0.15 * (df['esg_governance'] < df['esg_governance'].quantile(0.3)).astype(int) * 10  # Japan: Governance matters!
)

# Focus on latest year (2025)
latest_df = df[df['year'] == 2025].copy()

# Show top 10 high-risk companies
print(" Top 10 High-Risk Companies (2025):")
print(latest_df[['company_name', 'industry', 'region', 'esg_overall', 'esg_governance', 'risk_score']]
      .sort_values('risk_score', ascending=False)
      .head(10))

 Top 10 High-Risk Companies (2025):
      company_name        industry         region  esg_overall  \
10846  Company_986          Retail  Latin America         33.3   
935    Company_174  Transportation         Africa         21.9   
1683   Company_235      Healthcare    Middle East         37.5   
9064    Company_84  Consumer Goods    Middle East         18.1   
1078   Company_186   Manufacturing    Middle East         28.9   
5676   Company_562  Transportation         Africa         25.7   
8261   Company_774       Utilities         Africa         30.7   
5500   Company_548          Retail    Middle East         29.1   
1309   Company_204          Retail    Middle East         27.6   
2508   Company_302          Energy  Latin America         16.1   

       esg_governance  risk_score  
10846            29.9   35.123874  
935               8.7   32.849225  
1683              8.0   32.763375  
9064             12.8   32.759690  
1078             13.2   32.752281  
5676             27.0

In [11]:
# =============================================
# STEP 4: Simulate News Sentiment Proxy
# =============================================

# Define negative news triggers
df['news_negative_flag'] = (
    (df['risk_score'] > 30) |
    (df['esg_governance'] < 20) |
    ((df['profit_margin'] < 5) & (df['esg_trend'] < 0))
).astype(int)

# Convert to sentiment score (-1.0 = negative, +0.5 = neutral/positive)
df['news_sentiment'] = df['news_negative_flag'].apply(lambda x: -1.0 if x == 1 else 0.5)

# For Japan relevance: extra penalty if region = Asia AND governance < 30
df.loc[(df['region'] == 'Asia') & (df['esg_governance'] < 30), 'news_sentiment'] = -1.0

# Show top 5 companies with simulated negative news (2025)
latest_df = df[df['year'] == 2025].copy()
print(" Top 5 Companies with Simulated Negative News (2025):")
print(latest_df[latest_df['news_sentiment'] == -1.0]
      [['company_name', 'industry', 'region', 'esg_overall', 'esg_governance', 'risk_score', 'news_sentiment']]
      .sort_values('risk_score', ascending=False)
      .head())

 Top 5 Companies with Simulated Negative News (2025):
      company_name        industry         region  esg_overall  \
10846  Company_986          Retail  Latin America         33.3   
935    Company_174  Transportation         Africa         21.9   
1683   Company_235      Healthcare    Middle East         37.5   
9064    Company_84  Consumer Goods    Middle East         18.1   
1078   Company_186   Manufacturing    Middle East         28.9   

       esg_governance  risk_score  news_sentiment  
10846            29.9   35.123874            -1.0  
935               8.7   32.849225            -1.0  
1683              8.0   32.763375            -1.0  
9064             12.8   32.759690            -1.0  
1078             13.2   32.752281            -1.0  


In [12]:
# =============================================
# STEP 3: Risk Segmentation via K-Means Clustering (Japan Focus)
# =============================================

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Focus on 2025 data
cluster_df = df[df['year'] == 2025].copy()

# Recalculate industry avg ESG for 2025 (needed for features)
cluster_df['industry_avg_esg'] = cluster_df.groupby('industry')['esg_overall'].transform('mean')
cluster_df['industry_avg_gov'] = cluster_df.groupby('industry')['esg_governance'].transform('mean')

# Select business-relevant features for clustering
features = [
    'esg_overall',
    'esg_governance',       # Critical for Japan
    'profit_margin',
    'risk_score'            # Your advanced risk score from Step 2
]

# Handle missing values (should be none, but safe)
X = cluster_df[features].fillna(0)

# Standardize features (required for K-Means)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply K-Means (3 clusters)
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_labels = kmeans.fit_predict(X_scaled)

# Add cluster labels to dataframe
cluster_df['risk_cluster'] = cluster_labels

# Label clusters by average risk_score (lowest → highest)
cluster_risk = cluster_df.groupby('risk_cluster')['risk_score'].mean().sort_values()
cluster_mapping = {
    cluster_risk.index[0]: 'Green Leader',
    cluster_risk.index[1]: 'Yellow Watch',
    cluster_risk.index[2]: 'Red Zone'
}
cluster_df['risk_segment'] = cluster_df['risk_cluster'].map(cluster_mapping)

# Merge back to main df
df = df.merge(cluster_df[['company_name', 'risk_segment']], on='company_name', how='left')

# Show segment distribution
print(" Risk Segment Distribution (2025):")
print(df[df['year'] == 2025]['risk_segment'].value_counts())

# Show Red Zone companies in Asia (Japan-relevant)
asia_red_zone = df[
    (df['year'] == 2025) & 
    (df['region'] == 'Asia') & 
    (df['risk_segment'] == 'Red Zone')
]
print(f"\n🇯🇵 Red Zone Companies in Asia (Japan Proxy): {len(asia_red_zone)}")
if len(asia_red_zone) > 0:
    print(asia_red_zone[['company_name', 'industry', 'esg_governance', 'risk_score']].head())
else:
    print("No Asia companies in Red Zone — great ESG performance!")

 Risk Segment Distribution (2025):
risk_segment
Yellow Watch    376
Green Leader    318
Red Zone        306
Name: count, dtype: int64

🇯🇵 Red Zone Companies in Asia (Japan Proxy): 45
     company_name       industry  esg_governance  risk_score
275    Company_12        Finance            30.3   25.513019
407   Company_130  Manufacturing            53.3   27.529312
715   Company_156     Technology            14.4   22.893702
913   Company_172         Energy            34.0   20.810685
1166  Company_193  Manufacturing            23.8   22.390512


C:\Users\anith\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


In [13]:
# =============================================
# STEP 6: Export Final Dataset for Power BI
# =============================================

# Select only 2025 data (most relevant for dashboard)
final_df = df[df['year'] == 2025].copy()

# Select key columns for Power BI
columns_for_powerbi = [
    'company_name',
    'industry',
    'region',
    'is_japan_relevant',  # 1 if Asia, 0 otherwise
    'esg_overall',
    'esg_environmental',
    'esg_social',
    'esg_governance',
    'profit_margin',
    'revenue',
    'risk_score',
    'risk_segment',
    'news_sentiment'
]

final_df = final_df[columns_for_powerbi]

# Save to Excel (Power BI reads Excel perfectly)
final_df.to_excel("ESG_Risk_Analysis_Final.xlsx", index=False)

print(" Exported final dataset to 'ESG_Risk_Analysis_Final.xlsx'")
print(f" Total companies in 2025: {len(final_df)}")
print(f" Red Zone count: {len(final_df[final_df['risk_segment'] == 'Red Zone'])}")
print(f"🇯🇵 Japan-relevant (Asia) count: {final_df['is_japan_relevant'].sum()}")

 Exported final dataset to 'ESG_Risk_Analysis_Final.xlsx'
 Total companies in 2025: 1000
 Red Zone count: 306
🇯🇵 Japan-relevant (Asia) count: 152


In [14]:
# =============================================
# JAPAN ESG CONTEXT: Why This Matters
# =============================================
print("🇯🇵 ACCENTURE JAPAN CONTEXT:")
print("- Tokyo Stock Exchange (TSE) requires ESG disclosures since 2022")
print("- 80% of TOPIX companies now publish ESG reports")
print("- Governance (G) is the #1 investor concern in Japan")
print("- Our model penalizes low governance — aligned with Japan priorities")
print(f"\n In our data: {final_df['is_japan_relevant'].sum()} Asia companies")
print(f" {len(final_df[(final_df['is_japan_relevant'] == 1) & (final_df['risk_segment'] == 'Red Zone')])} Japanese firms in Red Zone")

🇯🇵 ACCENTURE JAPAN CONTEXT:
- Tokyo Stock Exchange (TSE) requires ESG disclosures since 2022
- 80% of TOPIX companies now publish ESG reports
- Governance (G) is the #1 investor concern in Japan
- Our model penalizes low governance — aligned with Japan priorities

 In our data: 152 Asia companies
 45 Japanese firms in Red Zone


In [15]:
# =============================================
# STEP 7: Add Client-Ready Recommendations (FIXED)
# =============================================

# First, calculate industry average ESG for 2025 (needed for recommendations)
final_df['industry_avg_esg'] = final_df.groupby('industry')['esg_overall'].transform('mean')

# Now define the recommendation function
def generate_recommendation(row):
    if row['risk_segment'] == 'Red Zone':
        gap = row['industry_avg_esg'] - row['esg_overall']
        if gap > 5:
            return f"Close ESG gap: Improve by {gap:.1f} points to match industry"
        elif row['esg_governance'] < 40:
            return "Urgent: Strengthen board oversight & disclosure (Governance < 40)"
        else:
            return "Conduct ESG audit; simulate improvement scenarios"
    elif row['risk_segment'] == 'Yellow Watch':
        return "Benchmark vs peers; address ESG trend decline"
    else:
        return "Leverage ESG leadership for investor relations"

# Apply the function
final_df['recommendation'] = final_df.apply(generate_recommendation, axis=1)

# Save the final enriched dataset
final_df.to_excel("ESG_Risk_Analysis_Final.xlsx", index=False)

print(" Added client-ready recommendations to Excel!")
print(f" File saved: ESG_Risk_Analysis_Final.xlsx")

 Added client-ready recommendations to Excel!
 File saved: ESG_Risk_Analysis_Final.xlsx


In [26]:
final_df.to_excel("ESG_Risk_Analysis_Final.xlsx", index=False)

In [16]:
# Export ALL YEARS for Power BI (for trend analysis)
df_all_years = df.copy()  # Keep all years

# Save as a separate file for Power BI
df_all_years.to_excel("ESG_Risk_Analysis_All_Years.xlsx", index=False)

print("Exported full dataset with all years for trend analysis.")

Exported full dataset with all years for trend analysis.


In [18]:
# Add risk driver components (all positive = higher risk)
final_df['risk_from_gap'] = (final_df['industry_avg_esg'] - final_df['esg_overall']).clip(lower=0)

final_df['risk_from_financial'] = final_df['financial_stress'].astype(int) * 10

# Re-export
final_df.to_excel("ESG_Risk_Analysis_Final.xlsx", index=False)

KeyError: 'financial_stress'